In [1]:
from __future__ import annotations
from typing import Tuple, List, Dict, KeysView, Iterable
from random import uniform
from gurobipy import Model, tupledict, GurobiError, GRB

In [2]:
Vertex = int
Arc = Tuple[Vertex, Vertex]
Tour = List[Vertex]

class TSPIntance:
    n: int
    x: List[float]
    y: List[float]
    cost: Dict[Arc, float]

    def __init__(self, x: List[float], y: List[float]):
        assert len(x) == len(y), "X and Y coordinate lists must have the same length"

        self.n = len(x)
        self.x = x
        self.y = y
        self.cost = {
            (i, j): ((self.x[i] - self.x[j])**2 + (self.y[i] - self.y[j])**2)**0.5
            for i in self.vertices()
            for j in self.vertices()
            if i != j
        }

    def vertices(self) -> Iterable[Vertex]:
        return range(self.n)
    
    def arcs(self) -> KeysView:
        return self.cost.keys()

    @staticmethod
    def random(n: int) -> TSPIntance:
        x = [uniform(0, 10) for _ in range(n)]
        y = [uniform(0, 10) for _ in range(n)]
        return TSPIntance(x=x, y=y)

In [3]:
class TSPSolution:
    tour: Tour
    cost: float

    def __init__(self, tour: Tour, **kwargs):
        assert 'cost' in kwargs or 'instance' in kwargs, \
            "You must pass the tour cost or a TSP instance to compute it"

        if 'cost' in kwargs:
            self.cost = kwargs.get('cost')
        elif 'instance' in kwargs:
            tsp = kwargs.get('instance')
            self.cost = sum(
                tsp.cost[i, j]
                for i in tour[:-1]
                for j in tour[1:]
            )

        self.tour = tour

    def __str__(self) -> str:
        return "[" + ', '.join(map(str, self.tour)) + f"] - Cost: {self.cost:.2f}"

In [4]:
class BranchAndCutIntegerSolver:
    tsp: TSPIntance
    m: Model
    x: tupledict

    def __init__(self, tsp: TSPIntance):
        self.tsp = tsp
        self.m = Model()
        self.x = self.m.addVars(self.tsp.arcs(), obj=self.tsp.cost, vtype=GRB.BINARY, name='x')
        self.__build_model()

    def __build_model(self) -> None:
        self.m.addConstrs(self.x.sum(i, '*') == 1 for i in self.tsp.vertices())
        self.m.addConstrs(self.x.sum('*', i) == 1 for i in self.tsp.vertices())

    def solve(self) -> TSPSolution:
        self.m.setParam(GRB.Param.LazyConstraints, 1)
        self.m.optimize(lambda _, where: self.__separate(where=where))

        if self.m.Status != GRB.OPTIMAL:
            raise RuntimeError("Could not solve TSP model to optimality")
        
        return TSPSolution(tour=self.__tour_starting_at(0), cost=self.m.ObjVal)
    
    def __separate(self, where: int) -> None:
        if where != GRB.Callback.MIPSOL:
            return
        
        remaining = set(self.tsp.vertices())

        while len(remaining) > 0:
            # Get the first vertex of the set
            start = next(iter(remaining))
            subtour = self.__tour_starting_at(start)

            if len(subtour) == self.tsp.n:
                # Feasible tour visiting all vertices
                return
            
            self.__add_sec_for(subtour)

            remaining -= set(subtour)

    def __tour_starting_at(self, i: Vertex) -> Tour:
        tour = [i]
        current = self.__next_vertex(i=i)

        while current != i:
            tour.append(current)
            current = self.__next_vertex(current)

        return tour

    def __next_vertex(self, i: Vertex) -> Vertex:
        for j in self.tsp.vertices():
            if j == i:
                continue

            try:
                # When in a callback
                x = self.m.cbGetSolution(self.x[i,j])
            except GurobiError:
                # When optimisation is over
                x = self.x[i,j].X

            if x > 0.5:
                return j
            
        raise RuntimeError(f"Vertex {i} has no successor!")
    
    def __add_sec_for(self, subtour: Tour) -> None:
        print("Adding subtour for [" + ', '.join(map(str, subtour)) + "]")
        self.m.cbLazy(
            sum(
                self.x[i, j]
                for i, j in self.tsp.arcs()
                if i in subtour and j not in subtour
            ) >= 1
        )

In [5]:
tsp = TSPIntance.random(n=20)
solver = BranchAndCutIntegerSolver(tsp=tsp)
solution = solver.solve()

Set parameter Username
Academic license - for non-commercial use only - expires 2024-11-26
Set parameter LazyConstraints to value 1
Gurobi Optimizer version 11.0.0 build v11.0.0rc2 (mac64[x86] - Darwin 21.6.0 21H1320)

CPU model: Intel(R) Core(TM) i7-6700K CPU @ 4.00GHz
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 40 rows, 380 columns and 760 nonzeros
Model fingerprint: 0x43e9c8be
Variable types: 0 continuous, 380 integer (380 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [6e-01, 1e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Adding subtour for [0, 1, 9, 2, 3, 7]
Adding subtour for [4, 17, 11, 8, 5, 10, 14]
Adding subtour for [6, 18, 16, 15]
Adding subtour for [12, 13, 19]
Presolve time: 0.00s
Presolved: 40 rows, 380 columns, 760 nonzeros
Variable types: 0 continuous, 380 integer (380 binary)
Adding subtour for [0, 15]
Adding subtour for [1, 10]
Adding subtour for [2

In [6]:
print(solution)

[0, 4, 16, 2, 12, 3, 18, 9, 13, 19, 14, 17, 6, 15, 7, 11, 1, 10, 8, 5] - Cost: 36.06
